<h1>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

importing libraries

In [2]:
import pandas as pd
import numpy as np
from urllib import request
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
from bs4 import BeautifulSoup
print('Libraries imported')

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: lxml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: html5lib in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: webencodings in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: setuptools>=18.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Libraries imported


<b>Loading webpage from Wikipedia

In [3]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [7]:
page = request.Request(wiki)
Wiki = request.urlopen(page).read().decode('utf-8')

<b>Parsing webpage scraping table

In [9]:
soup = BeautifulSoup(Wiki, 'lxml')

In [25]:
match = soup.find('div', class_='mw-parser-output')

In [71]:
match2  = match.table.tbody

<b>Taking the Column names

In [72]:
col = match2.find_all('th')

In [85]:
col

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
 </th>]

In [168]:
column = []
for i,j in enumerate(col): column.append(col[i].text.strip())

In [169]:
column

['Postcode', 'Borough', 'Neighbourhood']

In [246]:
df_toronto = pd.DataFrame(columns = column)

<b> Filling up the data

In [97]:
data = match2.find_all('td')

In [370]:
borough = None
neigh = None
for i in range(int(len(data)/3)):
    k = i*3
    if not (data[k+1].text) == 'Not assigned': 
        borough = str(data[k+1].text)
    if not (data[k+2].text.strip()) == 'Not assigned': 
        neigh = str(data[k+2].text.strip())
    df_toronto.loc[i] = pd.Series({column[0]:data[k].text, column[1]:borough, column[2]:neigh})

<b>Dropping not assigned boroughs

In [371]:
df_toronto.dropna(inplace=True)

In [383]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


<b>Merging same Post Code

In [362]:
df_t1 = df_toronto.groupby(df_toronto['Postcode'])['Neighbourhood'].apply(', '.join).reset_index()
df_t1.drop('Postcode', axis=1, inplace=True)

In [363]:
df_t2 = df_toronto.groupby('Postcode').agg({'Borough':'first'}).reset_index()

In [364]:
df_T = pd.concat([df_t2, df_t1], axis=1)

<b>Results

In [367]:
df_T.shape

(103, 3)

In [382]:
df_T.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<H2> Assignment part 2

<b>Installing geocoder

In [374]:
!pip install geocoder
import geocoder # import geocoder

    100% |████████████████████████████████| 102kB 6.3MB/s ta 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

<b> Verifiing if its work

In [379]:
# initialize your variable to None
lat_lng_coords = None
postal_code = 'M6N'
counter = 0
# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    counter += 1  
    if counter >= 200:
        break
    
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(postal_code, latitude, longitude, counter)

TypeError: 'NoneType' object is not subscriptable

<b> Didnt work, so let`s use the csv file.

<b> Reading the CSV into Pandas DataFrame

In [394]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<b>Sorting by Postal Code

In [397]:
#This is to prevent mistakes when dataframes will merged
df_geo.sort_values('Postal Code', inplace= True)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<b>Merging the first DF with the second

In [411]:
df_geo.drop('Postal Code', axis = 1,inplace=True)

In [412]:
df_Toronto = pd.concat([df_T,df_geo], axis=1)

In [414]:
df_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
